## Exercise

Add a tool to set the price of a ticket!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Hopefully this hardly needs to be stated! You now have the ability to give actions to your LLMs. This Airline Assistant can now do more than answer questions - it could interact with booking APIs to make bookings!</span>
        </td>
    </tr>
</table>

In [107]:
import os 
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

load_dotenv(override=True)

openai_api_key = os.getenv("OPEN_API_KEY")
MODEL = "gpt-4.1-mini"
openai = OpenAI()

system_message = """
You are an airline booking AI agent called OpenFlight.
You should give short, courteous answers, not more than 1 sentence.
Always be accurate. 
"""

PRICE_DB = "prices.db"
CHAT_HISTORY_DB = "chat_history.db"


In [145]:
# Define SQLite connection query
def get_seed_data():
    ticket_price = [
        ("london", 799),
        ("paris", 899)]
    history = [
        ("DUMMY", "user", "I am diduboyz"),
        ("DUMMY", "assistant", "Hello! How can I assist you with your flight today")
    ]
    seed_data = {
       "ticket_price": ticket_price,
       "chat_history": history
    }
    return seed_data

def initialize_tables():
    seed_data = get_seed_data()
    with sqlite3.connect(PRICE_DB) as conn:
        if not is_table_exist(db_path=PRICE_DB, table_name="prices"):
            price_data = seed_data["ticket_price"]
            cursor = conn.cursor()
            cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
            conn.commit()
            for city, price in price_data:
                set_ticket_price(city=city, price=price)

    with sqlite3.connect(CHAT_HISTORY_DB) as conn:
        if not is_table_exist(db_path=CHAT_HISTORY_DB, table_name="chatHistory"):
            print("chatHistory not exist")
            chat_history = seed_data["chat_history"]
            print(chat_history)
            cursor = conn.cursor()
            cursor.execute('CREATE TABLE IF NOT EXISTS chatHistory (username TEXT, role TEXT, content TEXT)')
            conn.commit()
            for user_name, role, content in chat_history:
                set_chat_history(user_name=user_name, role=role, content=content)


def is_table_exist(db_path, table_name):
    with sqlite3.connect(database=db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("""SELECT name FROM sqlite_master WHERE type = 'table' AND name = ?""", (table_name,))
        return cursor.fetchone() is not None

def get_ticket_price(city):
    print(f"Database Tool called : get_ticket_price for {city}", flush=True)
    with sqlite3.connect(database=PRICE_DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return result[0] if result else ""

def set_ticket_price(city, price):
    print(f"Database Tool called : set_ticket_price for {city} with {price}", flush=True)
    with sqlite3.connect(database=PRICE_DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?,?) ON CONFLICT (city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

def get_chat_history(user_name):
    print(f"Database Tool called: retrieve_chat_history for {user_name} ", flush=True)
    with sqlite3.connect(database=CHAT_HISTORY_DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT role, content FROM chatHistory WHERE username = ?', (user_name,))
        result = cursor.fetchall()
        return result if result else []

def set_chat_history(user_name, role, content):
    print(f"Database Tool called: set_chat_history for {user_name}, {role}, {content} ", flush=True)
    with sqlite3.connect(database=CHAT_HISTORY_DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO chatHistory (username, role, content) VALUES (?, ?, ?)', (user_name, role, content))
        conn.commit()



In [180]:
# Define call functions
def retrieve_ticket_price(destination_city):
    print(f"retrieve_ticket_price destination_city:{destination_city}", flush=True)
    price = get_ticket_price(city=destination_city.lower())
    print(f"result price: {price}")
    return f"the price of a ticket to {destination_city} is {price}" if price != "" else "Unknown ticket price"

# dictionary structures for call functions
def get_tools():
    tool_functions = [
        {
            "type": "function",
            "function": {
                "name" : "retrieve_ticket_price",
                "description": "Get ticket price",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination_city" : {
                            "type": "string",
                            "description": "Destination City"
                        }},
                    "required": ["destination_city"],
                    "additionalProperties": False
                } 
            }
        },
        {
            "type": "function",
            "function": {
                "name" : "set_ticket_price",
                "description": "Set ticket price",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination_city" : {
                            "type": "string",
                            "description": "Destination City"
                        },
                        "price" : {
                            "type": "number",
                            "description": f"Price of the ticket"
                        }
                        },
                    "required": ["destination_city", "price"],
                    "additionalProperties": False
                } 
            }
        },
    ]
    return tool_functions

In [185]:
# handle_tool function
def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "retrieve_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = retrieve_ticket_price(city)
            response = {
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            }
            responses.append(response)
            
        elif tool_call.function.name == "set_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price = arguments.get('price')
            result = set_ticket_price(city, price)
            response = {
                "role": "tool",
                "content": "Record successfully added to table" if result is None else result,
                "tool_call_id": tool_call.id
            }
            responses.append(response)
        
    return responses
        
    
def chat(message, history):
    user_name = "DUMMY"  #need to find a way to get user_name
    history_data = get_chat_history(user_name)
    history = []
    for role, content in history_data:
        history.extend([{"role":role, "content":content}])
        
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message }]
    print(f"message: {messages}")
    
    tools = get_tools()
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    choice = response.choices[0]
    print(f"choice: {choice}")
    
    while choice.finish_reason == "tool_calls":
        responses = handle_tool_call(choice.message)
        print(f"results: {responses}")
        messages.append(choice.message)
        messages.extend(responses)
        
        print(f"messages: {messages}")
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        choice = response.choices[0]

    print(f"set chat history: {user_name}, {choice.message.role}, {choice.message.content}")
    set_chat_history(user_name, choice.message.role, choice.message.content)
    return choice.message.content

In [ ]:
initialize_tables()

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7933
* To create a public link, set `share=True` in `launch()`.


Database Tool called: retrieve_chat_history for DUMMY 
message: [{'role': 'system', 'content': '\nYou are an airline booking AI agent called OpenFlight.\nYou should give short, courteous answers, not more than 1 sentence.\nAlways be accurate. \n'}, {'role': 'user', 'content': 'I am diduboyz'}, {'role': 'assistant', 'content': 'Hello! How can I assist you with your flight today'}, {'role': 'assistant', 'content': 'Could you please clarify how I can assist you with your flight booking?'}, {'role': 'assistant', 'content': 'The ticket price to London is $799, which is above $500, so I did not check the price for Paris.'}, {'role': 'assistant', 'content': "The ticket price to London is $799; since it's less than $1000, the ticket price to Paris is $899."}, {'role': 'assistant', 'content': 'The ticket price to London is $799.'}, {'role': 'assistant', 'content': 'Please provide the price you would like to set for the ticket to Singapore.'}, {'role': 'user', 'content': 'set singapore price to 